Installation de Shap-E et de ses dépendances.

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install git+https://github.com/openai/shap-e.git
!pip install flask cloudflared trimesh tqdm imageio[ffmpeg] matplotlib


Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/openai/shap-e.git to /tmp/pip-req-build-h1clchjx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/shap-e.git /tmp/pip-req-build-h1clchjx
  Resolved https://github.com/openai/shap-e.git to commit 50131012ee11c9d2617f3886c10f000d3c7a3b43
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-_xcgfxfh/clip_dec5097ef25a4aa985a5eaff5b06746e
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-_xcgfxfh/clip_dec5097ef25a4aa985a5eaff5b06746e
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


Installation de Flask Cors

In [ ]:
!pip install flask-cors


notre serveur

In [ ]:
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import torch, time, trimesh
from shap_e.models.download import load_model, load_config
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.util.notebooks import decode_latent_mesh

app = Flask(__name__)
CORS(app)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json()
    prompt = data.get("prompt", "a simple cube")

    latents = sample_latents(
        batch_size=1,
        model=model,
        diffusion=diffusion,
        guidance_scale=15.0,
        model_kwargs=dict(texts=[prompt]),
        progress=False,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    mesh = decode_latent_mesh(xm, latents[0]).tri_mesh()
    filename = f"model_{int(time.time())}.glb"
    tmesh = trimesh.Trimesh(vertices=mesh.verts, faces=mesh.faces)
    tmesh.export(filename)
    print(f"Modèle {filename} généré pour prompt: {prompt}")

    return jsonify({"status": "success", "file": filename})

@app.route("/<path:filename>")
def serve_file(filename):
    return send_from_directory(".", filename)

#  Démarrage
import threading
def run_server():
    app.run(host="0.0.0.0", port=5000)
threading.Thread(target=run_server).start()


/usr/local/lib/python3.12/dist-packages/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.12/dist-packages/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.12/dist-packages/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.12/dist-packages/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


Téléchargement et installation de cloudflared

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!cloudflared --version


(Reading database ... 125086 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.11.1) over (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...
cloudflared version 2025.11.1 (built 2025-11-07-16:59 UTC)


Lancement de notre API, Veuillez changer le lien sur vore App.jsx avec le lien généré

In [ ]:
!cloudflared tunnel --url http://localhost:5000 --no-autoupdate


2025-11-10T07:43:26Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-10T07:43:26Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-10T07:43:31Z INF +--------------------------------------------------------------------------------------------+
2025-11-10T07:43:31Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-10T07:43:31Z INF |  https://gmt-equilibrium-generous-friendship.trycloudf

INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 07:44:13] "OPTIONS /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 07:44:37] "POST /generate HTTP/1.1" 200 -


✅ Modèle model_1762760677.glb généré pour prompt: crystal shark


INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 07:44:37] "GET /model_1762760677.glb HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 07:44:57] "GET /model_1762760677.glb HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 07:45:27] "OPTIONS /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 07:45:51] "POST /generate HTTP/1.1" 200 -


✅ Modèle model_1762760751.glb généré pour prompt: simple cloud


INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 07:45:52] "GET /model_1762760751.glb HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 07:46:14] "GET /model_1762760751.glb HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 08:02:14] "OPTIONS /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 08:02:38] "POST /generate HTTP/1.1" 200 -


✅ Modèle model_1762761758.glb généré pour prompt: the moon


INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 08:02:39] "GET /model_1762761758.glb HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2025 08:02:52] "GET /model_1762761758.glb HTTP/1.1" 200 -
